In [1]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

In [2]:
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack
from textattack.transformations import WordSwap

from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset
import textattack

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import torch
from torch.utils.data import Dataset, DataLoader
#import pytorch_lightning as pl

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM

from sentence_transformers import SentenceTransformer, util
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup

import re
import sys
import random
from random import choices
import os
from itertools import chain
from string import punctuation
import time
import argparse

import csv
import json

import traceback
import gc
from enum import Enum 

import numpy as np
import pandas as pd
from statistics import mean

import matplotlib.pyplot as plt


# from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
# from trl.ppo import PPOTrainer
#from trl.core import build_bert_batch_from_txt

from IPython.core.display import Markdown,display, HTML, Latex
import qgrid

from verisci.covid import AbstractRetriever, RationaleSelector, LabelPredictor
from verisci.evaluate.lib.data import GoldDataset

from GPUtil import showUtilization as gpu_usage
import wandb

from tqdm.notebook import tqdm
tqdm.pandas()
import pickle

In [4]:
from sklearn import preprocessing


In [5]:
pd.set_option("display.precision", 2)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class ParaphraseTargetDirection(Enum):
    org_support_to_gen_refute = 0
    org_refute_to_gen_support = 1
    
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(37)    

In [8]:
PARAPHRASE_PROJECT_SETTINGS={
    'file_and_dirs': {
        'file_paraphrased_no_tune_all_model_full' : '../../dfs_generated/paraphrased/t5_no_fine_tune_generated_claim_all_model_df_full_1.pkl', # can be deleted
        'file_org_claims_by_scifact' : '../../dfs_generated/scifact/org_claim_ext_label_roberta_large_fever.pkl',
    },
    'config_scifact' : {
        'cls_model_name': '/home/computeruser/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact',
        'rationale_model_name': '/home/computeruser/research/nlp/fact_checking/my_work/scifact/model/rationale_roberta_large_fever_scifact',
        'loc_gold_ds_corpus' : '/home/computeruser/research/nlp/fact_checking/my_work/scifact/data/corpus.jsonl', 
        'loc_gold_ds_train' : '/home/computeruser/research/nlp/fact_checking/my_work/scifact/data/claims_train.jsonl', 
        'loc_gold_ds_dev' : '/home/computeruser/research/nlp/fact_checking/my_work/scifact/data/claims_dev.jsonl', 

    },
    
    
    'paraphrase_model' :
    {
        'list_potential_paraphrase_models' : 
            [
                {'model_name' : 'parasci_base_no_fine_tune' , 'model_path_or_url' : 'HelloRusk/t5-base-parasci', 'is_selected' : False},
                {'model_name' : 'parrot_base_no_fine_tune' , 'model_path_or_url' : 'prithivida/parrot_paraphraser_on_T5', 'is_selected' : False},
                {'model_name' : 'parrot_div_base_no_fine_tune' , 'model_path_or_url' : 'prithivida/parrot_paraphraser_on_T5', 'is_selected' : False},
                {'model_name' : 'pegasus_base_no_fine_tune' , 'model_path_or_url' : 'tuner007/pegasus_paraphrase', 'is_selected' : False},
                {'model_name' : 'paws_base_no_fine_tune' , 'model_path_or_url' : 'Vamsi/T5_Paraphrase_Paws', 'is_selected' : True},
                {'model_name' : 'tapaco_base_no_fine_tune' , 'model_path_or_url' : 'hetpandya/t5-base-tapaco', 'is_selected' : False},
                {'model_name' : 'sci_five_pubmed' , 'model_path_or_url' : 'razent/SciFive-large-Pubmed_PMC', 'is_selected' : False}
            ],
        't5_paraphrase_model_params':
        {
            'max_length':256,
            'do_sample':True,
            'top_k':50,
            'top_p': 0.99,
            'repetition_penalty':3.5,
            'early_stopping':True,
            'num_return_sequences':20
        }
    },
    
    'run_settings':
    {
        'PARAPHRASE_FT_TRAIN_SPLIT' : 0.1,
        'PARAPHRASE_FT_DATASET_DIRECTION' : ParaphraseTargetDirection.org_refute_to_gen_support,#ParaphraseTargetDirection.org_support_to_gen_refute,#ParaphraseTargetDirection.org_refute_to_gen_support,
        'NUM_OF_EPOCH_REQ_FT' : 2,
        'FILTER_BY' : 'TECH_TERMS',
        'SIMILARITY_THRESHOLD' : -100
        #'CUR_MODEL_NAME_PATHS' : (lambda: [_x['model_path_or_url'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True])(),
    },
}

CUR_NO_OF_EPOCH_FT = 0

## Scifact Functinos

### Load data

In [9]:
def get_claim_label_from_jsonl(dataset_jsonl):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = cur_claim.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name

            tmp_dic = {"claim" : claim_txt, "label" : claim_label}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [10]:
def get_claim_label_evidence_from_jsonl(dataset_jsonl, source):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = claim_train.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name
            
            list_rationales = []
            for i, sents in enumerate(evidence.rationales):
                list_rationales = [sent for i, sent in enumerate(ev_doc.sentences) if i in sents]

            tmp_dic = {"claim" : claim_txt, "label" : claim_label, "list_rationales" :list_rationales, "source" :source}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [11]:
ds_train = GoldDataset(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_corpus'],
                       PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_train'])
claim_train = ds_train.get_claim(39)
claim_train.pretty_print()

dic_train = get_claim_label_evidence_from_jsonl(ds_train, source = "train")

Example 39: A diminished ovarian reserve does not solely indicate infertility in an a priori non-infertile population.

Evidence sets:

####################

13497630: SUPPORTS
Set 0:
	- After adjusting for age, body mass index, race, current smoking status, and recent hormonal contraceptive use, women with low AMH values (<0.7 ng/mL [n = 84]) did not have a significantly different predicted probability of conceiving by 6 cycles of attempt (65%; 95% CI, 50%-75%) compared with women (n = 579) with normal values (62%; 95% CI, 57%-66%) or by 12 cycles of attempt (84% [95% CI, 70%-91%] vs 75% [95% CI, 70%-79%], respectively).
Set 1:
	- Women with high serum FSH values (>10 mIU/mL [n = 83]) did not have a significantly different predicted probability of conceiving after 6 cycles of attempt (63%; 95% CI, 50%-73%) compared with women (n = 654) with normal values (62%; 95% CI, 57%-66%) or after 12 cycles of attempt (82% [95% CI, 70%-89%] vs 75% [95% CI, 70%-78%], respectively).
Set 2:
	- Women

In [12]:
ds_valid = GoldDataset(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_corpus'],
                       PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_dev'])
claim_valid = ds_valid.get_claim(42)
claim_valid.pretty_print()

dic_valid = get_claim_label_evidence_from_jsonl(ds_valid, source = "dev")

Example 42: A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.

Evidence sets:

####################

18174210: REFUTES
Set 0:
	- Individuals homozygous for alpha(+)-thalassaemia have microcytosis and an increased erythrocyte count.
	- We estimated that the haematological profile in children homozygous for alpha(+)-thalassaemia reduces the risk of SMA during acute malaria compared to children of normal genotype (relative risk 0.52; 95% confidence interval [CI] 0.24-1.12, p = 0.09).   

Set 1:
	- CONCLUSIONS The increased erythrocyte count and microcytosis in children homozygous for alpha(+)-thalassaemia may contribute substantially to their protection against SMA.


In [13]:
df_claim_evid_label = pd.concat([pd.DataFrame(dic_train), pd.DataFrame(dic_valid)], ignore_index=True)

#df_claim_evid_label

In [14]:
df_claim_evid_label

,claim,label,list_rationales,source
0,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train
1,32% of liver transplantation programs required...,SUPPORTS,[Policies requiring discontinuation of methado...,train
2,40mg/day dosage of folic acid and 2mg/day dosa...,SUPPORTS,[CONCLUSION Treatment with high doses of folic...,train
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train
4,A T helper 2 cell (Th2) environment impedes di...,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train
...,...,...,...,...
768,Women with a higher birth weight are more like...,SUPPORTS,[Increased risk of breast cancer was noted wit...,dev
769,Women with a higher birth weight are more like...,SUPPORTS,[RESULTS We found that heavier birth weights w...,dev
770,aPKCz causes tumour enhancement by affecting g...,REFUTES,"[Taken together, this demonstrates that PKCζ i...",dev
771,cSMAC formation enhances weak ligand signalling.,SUPPORTS,[This conclusion was supported by experiments ...,dev


### Scifact Model

In [15]:
class ArgsScifact:
    def __init__(self, claim):
        self.claim = claim
        self.report_file = "/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/results/covid/report" #not needed
        self.n_documents = 100
        self.rationale_selection_method = "topk"
        self.output_format = "markdown"
        self.rationale_threshold = 0.5
        self.label_threshold = 0.5
        self.keep_nei = False
        self.full_abstract = True
        self.verbose = True
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        
        ##
class PretrainedModelsForScifact:
    def __init__(self, args):
        if args.device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(args.device)
            
        #self.rationale_selection_model = '/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/model/rationale_roberta_large_scifact'
        self.rationale_selection_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']
        self.label_prediction_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name']
        self.abstract_retriever = AbstractRetriever()
        self.rationale_selector = RationaleSelector(self.rationale_selection_model,
                                               args.rationale_selection_method,
                                               args.rationale_threshold,
                                               self.device)
        self.label_predictor = LabelPredictor(self.label_prediction_model,
                                         args.keep_nei,
                                         args.label_threshold,
                                         self.device)


In [16]:
args_sci = ArgsScifact("")

pretrained_models_config = PretrainedModelsForScifact(args_sci)

Some weights of the model checkpoint at /home/computeruser/research/nlp/fact_checking/my_work/scifact/model/rationale_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/computeruser/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.den

In [17]:
log_failed_claim = []
def inference(args, pretraind_models_config):

    try:
#         if args.verbose:
#         print("Retrieving abstracts. inference > ", args.claim)
        results = pretraind_models_config.abstract_retriever(args.claim, k=args.n_documents)
        if len(results) == 0:
            temp_dic = {'failed_in' : 'abstract retrival', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
        #print("abstract_retriever >> ", results)

#         if args.verbose:
#             print("Selecting rationales. inference > ", args.claim)
        results = pretraind_models_config.rationale_selector(args.claim, results)
        if len(results) == 0:
            temp_dic = {'failed_in' : 'Rationale selection', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
#         if args.verbose:
#             print("Label predictions. inference > ", args.claim)
        results = pretraind_models_config.label_predictor(args.claim, results)

        if len(results) == 0:
            temp_dic = {'failed_in' : 'Label Prediction', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
        results.sort(key=lambda r: r['label_confidence'], reverse=True)
        return results
    except Exception as e:
        print("Exception :: Inference cant retrive info for >> ", args.claim)
        print(sys.exc_info()[0])
        print(traceback.format_exc())
        temp_dic = {'failed_in' : sys.exc_info()[0], 'claim': args.claim}
        log_failed_claim.append(temp_dic)
        return []


In [18]:
def write_result(result, full_abstract):
    all_msg = ""
    all_msg = f"#### [{result['title']}]({result['url']}) \n"
    #print(msg, file=f)
    #all_msg = all_msg+msg
    ev_scores = [f"{x:0.2f}" for x in result["evidence_confidence"]]
    ev_scores = ", ".join(ev_scores)
    if result['label'].lower() == "support":
        msg = f"🟩 **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    elif result['label'].lower() == "refute":
        msg = f"🟥 **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    else:
        msg = f"⏺ **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    #print(msg, file=f)
    all_msg = all_msg+msg 
    
    for i, line in enumerate(result["abstract"]):
        # If we're showing the full abstract, show evidence in green.
        if full_abstract:
            if result['label'].lower() == "support":
                msg = (f"- <span style='color:green'>{line}</span>"
                       if i in result["evidence"]
                       else f"- {line}")
            elif result['label'].lower() == "refute":
                msg = (f"- <span style='color:red'>{line}</span>"
                       if i in result["evidence"]
                       else f"- {line}")                
            #print(msg, file=f)
            all_msg = all_msg+msg + " \n"
        else:
            if i in result["evidence"]:
                msg = f"- {line}"
                #print(msg, file=f)
                all_msg = all_msg+msg + " \n" 
    
    #print(file=f)
    #print(40 * "-", file=f)
    #print(file=f)
    all_msg = all_msg+msg 
    return all_msg + "\n"

In [19]:
def export(args, results):
    all_msg = ""
    claim = args.claim
    #report_file = args.report_file
    #f = open(f"{report_file}.md", "w")
    msg = f"### Claim \n > **{claim}** \n "
    #print(msg, file=f)
    #print(file=f)
    all_msg = all_msg +msg
    
    #support_confs = [], refute_confs = []
    confs = []
    for result in results:
        if result['label'].lower() == "support":
            tmp_dic = {'label' : 'Support', 'label_confidence' : result["label_confidence"], "no_of_evidence" : len(result['evidence_confidence'])}
            confs.append(tmp_dic)
        elif result['label'].lower() == "refute":
            tmp_dic = {'label' : 'Refute', 'label_confidence' : -result["label_confidence"], "no_of_evidence" : len(result['evidence_confidence'])}
            confs.append(tmp_dic)
        
    
    tpm_df = pd.DataFrame(confs)
    #HTML(tpm_df.style.bar(align='mid', color=['#d65f5f', '#5fba7d']))
    display(HTML(tpm_df.style.bar(subset=["label_confidence"], align='mid', color=['#ffa1a1', '#bfffcf']).render()))
    
    msg = "### Evidence \n "
    all_msg = all_msg +msg
    for result in results:
        cur_msg = write_result(result, args.full_abstract)
        all_msg = all_msg +cur_msg+"\n"

    return all_msg

In [20]:
claim_to_check = "ART substantially reduces infectiveness of HIV-positive people."#df_claim_evid_label.iloc[18, :]["claim"]
args_sci = ArgsScifact(claim_to_check)

#pretrained_models_config = pretrained_models_for_scifact(args_sci)

results_raw = inference(args_sci, pretrained_models_config)

if results_raw!= []:
    result_md = export(args_sci, results_raw)
    #result_md = export(args_sci, results_raw)
    display(Markdown(result_md))

/home/computeruser/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


,label,label_confidence,no_of_evidence
0,Support,0.69,3
1,Support,0.64,3
2,Support,0.54,3


### Claim 
 > **ART substantially reduces infectiveness of HIV-positive people.** 
 ### Evidence 
 #### [Autonomous Targeting of Infectious Superspreaders Using Engineered Transmissible Therapies](https://api.semanticscholar.org/10.1371/journal.pcbi.1002015) 
🟩 **Decision** : SUPPORT (score=0.69, evidence scores=0.11, 0.06, 0.01)
- Infectious disease treatments, both pharmaceutical and vaccine, face three universal challenges: the difficulty of targeting treatments to high-risk ‘superspreader’ populations who drive the great majority of disease spread, behavioral barriers in the host population (such as poor compliance and risk disinhibition), and the evolution of pathogen resistance. 
- Here, we describe a proposed intervention that would overcome these challenges by capitalizing upon Therapeutic Interfering Particles (TIPs) that are engineered to replicate conditionally in the presence of the pathogen and spread between individuals — analogous to ‘transmissible immunization’ that occurs with live-attenuated vaccines (but without the potential for reversion to virulence). 
- Building on analyses of HIV field data from sub-Saharan Africa, we construct a multi-scale model, beginning at the single-cell level, to predict the effect of TIPs on individual patient viral loads and ultimately population-level disease prevalence. 
- <span style='color:green'>Our results show that a TIP, engineered with properties based on a recent HIV gene-therapy trial, could stably lower HIV/AIDS prevalence by ∼30-fold within 50 years and could complement current therapies.</span> 
- <span style='color:green'>In contrast, optimistic antiretroviral therapy or vaccination campaigns alone could only lower HIV/AIDS prevalence by <2-fold over 50 years.</span> 
- The TIP's efficacy arises from its exploitation of the same risk factors as the pathogen, allowing it to autonomously penetrate superspreader populations, maintain efficacy despite behavioral disinhibition, and limit viral resistance. 
- <span style='color:green'>While demonstrated here for HIV, the TIP concept could apply broadly to many viral infectious diseases and would represent a new paradigm for disease control, away from pathogen eradication but toward robust disease suppression.</span> 
- <span style='color:green'>While demonstrated here for HIV, the TIP concept could apply broadly to many viral infectious diseases and would represent a new paradigm for disease control, away from pathogen eradication but toward robust disease suppression.</span>

#### [HIV: Biology to Treatment](https://api.semanticscholar.org/10.1007/978-981-32-9898-9_7) 
🟩 **Decision** : SUPPORT (score=0.64, evidence scores=0.67, 0.01, 0.37)
- AIDS is one of the most dreaded diseases of the twenty-first century caused by human immunodeficiency virus (HIV). 
- <span style='color:green'>Recently, there are reports which show decline in new infections due to better access to anti-retroviral drugs.</span> 
- <span style='color:green'>Still on a daily basis, ~2356 new HIV infections are being reported globally.</span> 
- New treatments and anti-HIV drugs are being continuously developed with the aim to control and cure AIDS. 
- The anti-HIV drugs that are in use usually target HIV entry and replication inside the host cells. 
- <span style='color:green'>However, these drugs are only partially effective in slowing the rate of HIV replication.</span> 
- Nevertheless, the virus manages to replicate at much slower rates even when anti-retroviral treatment is ongoing. 
- The HIV seropositives who are on anti-retroviral treatment for long periods of time are now developing different kinds of other complications including neuroAIDS. 
- The latest development in HIV therapy is a novel kind of bone marrow transplantation from donors who have a homozygous mutation in CCR5 gene. 
- The latest development in HIV therapy is a novel kind of bone marrow transplantation from donors who have a homozygous mutation in CCR5 gene.

#### [Human Immunodeficiency Virus-Associated Diarrhea: Still an Issue in the Era of Antiretroviral Therapy](https://api.semanticscholar.org/10.1007/s10620-015-3615-y) 
🟩 **Decision** : SUPPORT (score=0.54, evidence scores=0.01, 0.99, 0.02)
- <span style='color:green'>Over half of patients with human immunodeficiency virus (HIV) experience diarrhea that contributes negatively to quality of life and adherence to antiretroviral therapy (ART).</span> 
- Opportunistic infectious agents that cause diarrhea in patients with HIV span the array of protozoa, fungi, viruses, and bacteria. 
- <span style='color:green'>With global use of ART, the incidence of diarrhea because of opportunistic infections has decreased; however, the incidence of noninfectious diarrhea has increased.</span> 
- <span style='color:green'>The etiology of noninfectious diarrhea in patients with HIV is multifactorial and includes ART-associated diarrhea and gastrointestinal damage related to HIV infection (i.e., HIV enteropathy).</span> 
- A basic algorithm for the diagnosis of diarrhea in patients with HIV includes physical examination, a review of medical history, assessment of HIV viral load and CD4+ T cell count, stool microbiologic assessment, and endoscopic evaluation, if needed. 
- For patients with negative diagnostic results, the diagnosis of noninfectious diarrhea may be considered. 
- Pharmacologic options for the treatment of noninfectious diarrhea are primarily supportive; however, the use of many unapproved agents is based on unstudied and anecdotal information. 
- In addition, these agents can be associated with treatment-limiting adverse events (AEs), such as drug–drug interactions with ART regimens, abuse liability, and additional gastrointestinal AEs. 
- Currently, crofelemer, an antisecretory agent, is the only therapy approved in the USA for the symptomatic relief of noninfectious diarrhea in patients with HIV on ART. 
- Currently, crofelemer, an antisecretory agent, is the only therapy approved in the USA for the symptomatic relief of noninfectious diarrhea in patients with HIV on ART.



In [21]:
#print(pretraind_models_config)

## Data Prep


In [22]:
le_ground = preprocessing.LabelEncoder()
le_ground.fit(df_claim_evid_label.label)
df_claim_evid_label['label_enc'] = le_ground.transform(df_claim_evid_label.label)

In [23]:
df_claim_evid_label

,claim,label,list_rationales,source,label_enc
0,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,0
1,32% of liver transplantation programs required...,SUPPORTS,[Policies requiring discontinuation of methado...,train,1
2,40mg/day dosage of folic acid and 2mg/day dosa...,SUPPORTS,[CONCLUSION Treatment with high doses of folic...,train,1
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,1
4,A T helper 2 cell (Th2) environment impedes di...,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train,0
...,...,...,...,...,...
768,Women with a higher birth weight are more like...,SUPPORTS,[Increased risk of breast cancer was noted wit...,dev,1
769,Women with a higher birth weight are more like...,SUPPORTS,[RESULTS We found that heavier birth weights w...,dev,1
770,aPKCz causes tumour enhancement by affecting g...,REFUTES,"[Taken together, this demonstrates that PKCζ i...",dev,0
771,cSMAC formation enhances weak ligand signalling.,SUPPORTS,[This conclusion was supported by experiments ...,dev,1


In [24]:
le_name_mapping = dict(zip(le_ground.classes_, le_ground.transform(le_ground.classes_)))
print(le_name_mapping)

{'REFUTES': 0, 'SUPPORTS': 1}


In [25]:
tpl_claim_label_rationale = list(df_claim_evid_label[['claim', 'label_enc', 'label', 
                                                      'list_rationales', 'source']].itertuples(index=False, name=None))

In [26]:
tpl_claim_label_rationale

[('1 in 5 million in UK have abnormal PrP positivity.',
  0,
  'REFUTES',
  ['RESULTS Of the 32,441 appendix samples 16 were positive for abnormal PrP, indicating an overall prevalence of 493 per million population (95% confidence interval 282 to 801 per million).'],
  'train'),
 ('32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.',
  1,
  'SUPPORTS',
  ['Policies requiring discontinuation of methadone in 32% of all programs contradict the evidence base for efficacy of long-term replacement therapies and potentially result in relapse of previously stable patients.'],
  'train'),
 ('40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.',
  1,
  'SUPPORTS',
  ['CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease.   

In [27]:
ds_claim_label  = textattack.datasets.Dataset(tpl_claim_label_rationale, input_columns=(['claim']))

In [28]:
ds_claim_label

## TextFooler

In [46]:
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019

In [95]:
perturb_results_txt_fooler = []

In [21]:
model_txt_fooler = transformers.AutoModelForSequenceClassification.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])
tokenizer_txt_fooler = transformers.AutoTokenizer.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])

model_wrapper_txt_fooler = HuggingFaceModelWrapper(model_txt_fooler, tokenizer_txt_fooler)

Some weights of the model checkpoint at /home/computeruser/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
attack_args_fooler = AttackArgs(num_examples=-1)

attack_txt_fooler = TextFoolerJin2019.build(model_wrapper_txt_fooler)

attacker_txt_fooler = Attacker(attack_txt_fooler, ds_claim_label, attack_args_fooler)
attack_results_txt_fooler = attacker_txt_fooler.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/773 [00:00<00:53, 14.43it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
2022-01-04 23:07:58.274829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 23:07:58.275668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-01-04 23:07:58.275679: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for you

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (97%)]] --> [[[FAILED]]]

32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.


--------------------------------------------- Result 3 ---------------------------------------------
[[2 (61%)]] --> [[[SKIPPED]]]

40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.




[Succeeded / Failed / Skipped / Total] 0 / 2 / 3 / 5:   1%|          | 5/773 [00:43<1:52:36,  8.80s/it]

...

--------------------------------------------- Result 770 ---------------------------------------------
[[1 (71%)]] --> [[2 (55%)]]

[[Women]] with a higher birth weight are more likely to develop breast cancer later in life.

[[Dame]] with a higher birth weight are more likely to develop breast cancer later in life.


--------------------------------------------- Result 771 ---------------------------------------------
[[1 (87%)]] --> [[[SKIPPED]]]

aPKCz causes tumour enhancement by affecting glutamine metabolism.




[Succeeded / Failed / Skipped / Total] 105 / 388 / 279 / 772: 100%|█████████▉| 772/773 [1:09:47<00:05,  5.42s/it]

--------------------------------------------- Result 772 ---------------------------------------------
[[1 (96%)]] --> [[[FAILED]]]

cSMAC formation enhances weak ligand signalling.




[Succeeded / Failed / Skipped / Total] 105 / 389 / 279 / 773: 100%|██████████| 773/773 [1:09:53<00:00,  5.42s/it]

--------------------------------------------- Result 773 ---------------------------------------------
[[1 (95%)]] --> [[[FAILED]]]

mTORC2 regulates intracellular cysteine levels through xCT inhibition.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 105    |
| Number of failed attacks:     | 389    |
| Number of skipped attacks:    | 279    |
| Original accuracy:            | 63.91% |
| Accuracy under attack:        | 50.32% |
| Attack success rate:          | 21.26% |
| Average perturbed word %:     | 24.12% |
| Average num. words per input: | 12.24  |
| Avg num queries:              | 148.2  |
+-------------------------------+--------+


In [96]:
perturb_results_txt_fooler.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'TextFoolerJin2019') for cur_result in attack_results_txt_fooler] )

In [97]:
df_txt_fooler = pd.DataFrame(perturb_results_txt_fooler, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [98]:
df_txt_fooler

,org_claim,gen_claim,num_of_query,attack_result,attack_method
0,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 million in UK have abnormal PrP positiv...,1,SkippedAttackResult,TextFoolerJin2019
1,32% of liver transplantation programs required...,32% of liver transplant plans requiring patien...,219,FailedAttackResult,TextFoolerJin2019
2,40mg/day dosage of folic acid and 2mg/day dosa...,40mg/day dosage of folic acid and 2mg/day dosa...,1,SkippedAttackResult,TextFoolerJin2019
3,76-85% of people with severe mental disorder r...,76-85% of folks with hefty mental troubles att...,329,FailedAttackResult,TextFoolerJin2019
4,A T helper 2 cell (Th2) environment impedes di...,A T helper 2 cell (Th2) environment impedes di...,1,SkippedAttackResult,TextFoolerJin2019
...,...,...,...,...,...
769,Women with a higher birth weight are more like...,Dame with a higher birth weight are more likel...,56,SuccessfulAttackResult,TextFoolerJin2019
770,aPKCz causes tumour enhancement by affecting g...,aPKCz causes tumour enhancement by affecting g...,1,SkippedAttackResult,TextFoolerJin2019
771,cSMAC formation enhances weak ligand signalling.,cSMAC creating reinforces breakable ligand sig...,182,FailedAttackResult,TextFoolerJin2019
772,mTORC2 regulates intracellular cysteine levels...,mTORC2 regulates intracellular cysteine levell...,102,FailedAttackResult,TextFoolerJin2019


In [99]:
df_txt_fooler.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/textfooler_jin_2019.pkl')

In [100]:
df_txt_fooler.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/textfooler_jin_2019.csv')

## CheckList

In [28]:
from textattack.attack_recipes.checklist_ribeiro_2020 import CheckList2020

In [29]:
model_cklist = transformers.AutoModelForSequenceClassification.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])
tokenizer_cklist = transformers.AutoTokenizer.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])

model_wrapper_cklist = HuggingFaceModelWrapper(model_cklist, tokenizer_cklist)

Some weights of the model checkpoint at /home/computeruser/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
attack_args_cklist = AttackArgs(num_examples=-1)

attack_checklist = CheckList2020.build(model_wrapper_cklist)

attacker_checklist = Attacker(attack_checklist, ds_claim_label, attack_args_cklist)
attack_results_checklist = attacker_checklist.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/773 [00:00<00:51, 15.09it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapExtend
    (1): WordSwapContract
    (2): WordSwapChangeName
    (3): WordSwapChangeNumber
    (4): WordSwapChangeLocation
    )
  (constraints): 
    (0): RepeatModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (95%)]] --> [[[SKIPPED]]]

1 in 5 million in UK have abnormal PrP positivity.


2022-01-05 02:19:26,002 --------------------------------------------------------------------------------
2022-01-05 02:19:26,002 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2022-01-05 02:19:26,003  - The most current version of the model is automatically downloaded from there.
2022-01-05 02:19:26,003  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/r


2022-01-05 02:19:32,760 loading file /home/computeruser/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


[Succeeded / Failed / Skipped / Total] 0 / 2 / 2 / 4:   1%|          | 4/773 [00:09<29:32,  2.30s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (97%)]] --> [[[FAILED]]]

32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.


--------------------------------------------- Result 3 ---------------------------------------------
[[2 (61%)]] --> [[[SKIPPED]]]

40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.


--------------------------------------------- Result 4 ---------------------------------------------
[[1 (97%)]] --> [[[FAILED]]]

76-85% of people with severe mental disorder receive no treatment in low and middle income countries.




...

[Succeeded / Failed / Skipped / Total] 0 / 492 / 279 / 771: 100%|█████████▉| 771/773 [01:28<00:00,  8.68it/s]

--------------------------------------------- Result 769 ---------------------------------------------
[[1 (71%)]] --> [[[FAILED]]]

Women with a higher birth weight are more likely to develop breast cancer later in life.


--------------------------------------------- Result 770 ---------------------------------------------
[[1 (71%)]] --> [[[FAILED]]]

Women with a higher birth weight are more likely to develop breast cancer later in life.


--------------------------------------------- Result 771 ---------------------------------------------
[[1 (87%)]] --> [[[SKIPPED]]]

aPKCz causes tumour enhancement by affecting glutamine metabolism.




[Succeeded / Failed / Skipped / Total] 0 / 494 / 279 / 773: 100%|██████████| 773/773 [01:29<00:00,  8.68it/s]

--------------------------------------------- Result 772 ---------------------------------------------
[[1 (96%)]] --> [[[FAILED]]]

cSMAC formation enhances weak ligand signalling.


--------------------------------------------- Result 773 ---------------------------------------------
[[1 (95%)]] --> [[[FAILED]]]

mTORC2 regulates intracellular cysteine levels through xCT inhibition.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 494    |
| Number of skipped attacks:    | 279    |
| Original accuracy:            | 63.91% |
| Accuracy under attack:        | 63.91% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 12.24  |
| Avg num queries:              | 1.44   |
+-------------------------------+--------+



/home/computeruser/research/nlp/fact_checking/my_work/TextAttack/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/home/computeruser/anaconda3/envs/scifact/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [32]:
perturb_results_cklist = []

In [35]:
perturb_results_cklist.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'checklist_ribeiro_2020') for cur_result in attack_results_checklist] )

In [37]:
df_cklist= pd.DataFrame(perturb_results_cklist, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [38]:
df_cklist.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/checklist_ribeiro_2020.pkl')
df_cklist.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/checklist_ribeiro_2020.csv')

### BAE (BAE: BERT-Based Adversarial Examples)

In [29]:
from textattack.attack_recipes.bae_garg_2019 import BAEGarg2019

In [30]:
model_bae = transformers.AutoModelForSequenceClassification.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])
tokenizer_bae = transformers.AutoTokenizer.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])

model_wrapper_bae = HuggingFaceModelWrapper(model_bae, tokenizer_bae)

Some weights of the model checkpoint at /home/computeruser/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
attack_args_bae = AttackArgs(num_examples=-1)

attack_bae = BAEGarg2019.build(model_wrapper_bae)

attacker_bae = Attacker(attack_bae, ds_claim_label, attack_args_bae)
attack_results_bae = attacker_bae.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/773 [00:00<00:46, 16.46it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (95%)]] --> [[[SKIPPED]]]

1 in 5 million in UK have abnormal PrP positivity.




Using /tmp/tfhub_modules to cache modules.
2022-01-05 02:37:42.496753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-05 02:37:42.497378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-01-05 02:37:42.497388: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-05 02:37:42.497539: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (97%)]] --> [[[FAILED]]]

32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.


--------------------------------------------- Result 3 ---------------------------------------------
[[2 (61%)]] --> [[[SKIPPED]]]

40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.




[Succeeded / Failed / Skipped / Total] 0 / 2 / 3 / 5:   1%|          | 5/773 [00:24<1:01:53,  4.84s/it]

...

--------------------------------------------- Result 770 ---------------------------------------------
[[1 (71%)]] --> [[[FAILED]]]

Women with a higher birth weight are more likely to develop breast cancer later in life.


--------------------------------------------- Result 771 ---------------------------------------------
[[1 (87%)]] --> [[[SKIPPED]]]

aPKCz causes tumour enhancement by affecting glutamine metabolism.




[Succeeded / Failed / Skipped / Total] 179 / 314 / 279 / 772: 100%|█████████▉| 772/773 [1:02:58<00:04,  4.89s/it]

--------------------------------------------- Result 772 ---------------------------------------------
[[1 (96%)]] --> [[[FAILED]]]

cSMAC formation enhances weak ligand signalling.




[Succeeded / Failed / Skipped / Total] 179 / 315 / 279 / 773: 100%|██████████| 773/773 [1:03:11<00:00,  4.90s/it]

--------------------------------------------- Result 773 ---------------------------------------------
[[1 (95%)]] --> [[[FAILED]]]

mTORC2 regulates intracellular cysteine levels through xCT inhibition.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 179    |
| Number of failed attacks:     | 315    |
| Number of skipped attacks:    | 279    |
| Original accuracy:            | 63.91% |
| Accuracy under attack:        | 40.75% |
| Attack success rate:          | 36.23% |
| Average perturbed word %:     | 26.52% |
| Average num. words per input: | 12.24  |
| Avg num queries:              | 125.58 |
+-------------------------------+--------+


In [32]:
perturb_results_bae = []

In [33]:
perturb_results_bae.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'bae_garg_2019') for cur_result in attack_results_bae] )

In [35]:
df_bae= pd.DataFrame(perturb_results_bae, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [36]:
df_bae.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/bae_garg_2019.pkl')
df_bae.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/bae_garg_2019.csv')

## BERT-ATTACK 

In [29]:
from textattack.attack_recipes.bert_attack_li_2020 import BERTAttackLi2020

In [30]:
model_btk = transformers.AutoModelForSequenceClassification.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])
tokenizer_btk = transformers.AutoTokenizer.from_pretrained(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name'])

model_wrapper_btk = HuggingFaceModelWrapper(model_btk, tokenizer_btk)

Some weights of the model checkpoint at /home/computeruser/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
attack_args_btk = AttackArgs(num_examples=-1, enable_advance_metrics =True)

attack_btk = BERTAttackLi2020.build(model_wrapper_btk)

attacker_btk = Attacker(attack_btk, ds_claim_label, attack_args_btk)
attack_results_btk = attacker_btk.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/773 [00:00<00:51, 15.03it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.4
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (95%)]] --> [[[SKIPPED]]]

1 in 5 million in UK have abnormal PrP positivity.




Using /tmp/tfhub_modules to cache modules.
2022-01-05 13:01:10.740789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-05 13:01:10.741403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-01-05 13:01:10.741412: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-05 13:01:10.741577: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with

KeyboardInterrupt: 

In [ ]:
perturb_results_btk = []

In [ ]:
perturb_results_btk.extend([(cur_result.original_text(), cur_result.perturbed_text(), cur_result.num_queries, 
                                    str(cur_result.__class__.__name__), 'bert_attack_li_2020') for cur_result in attack_results_btk] )

In [ ]:
df_btk= pd.DataFrame(perturb_results_btk, columns = ['org_claim', 'gen_claim', 
                                                                    'num_of_query', 'attack_result', 'attack_method'])

In [ ]:
df_btk.to_pickle('../../dfs_generated/paraphrased/existing_literature_methods/bert_attack_li_2020.pkl')
df_btk.to_csv('../../dfs_generated/paraphrased/existing_literature_methods/bert_attack_li_2020.csv')

In [3]:
import transformers
print(transformers.__version__)

4.9.2


In [5]:
import huggingface_hub

In [6]:
print(huggingface_hub.__version__)

0.0.12


In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
import tensore